In [1]:
import pandas as pd
import numpy as np
import operator

In [28]:
submission_file_list = ['submission/[0.405]reverse_count_weight_canary_word_count_with_tag_2048_no_caption_0.40498.csv', 
                        'submission/[0.352]_plsr_800_pool5_to_bow_all_pca_2048.csv',
                        'submission/[0.31]submission_plsr_800_pool5_to_word2vec_n_v_ajd_300.csv',
                        'submission/[0.267]PLSR_100_BOW_all_pca_2048_tags_80_cos_0.26727.csv',
                        'submission/[0.23]MLPR_image_1000_des_BOW_all_23.csv',
                        'submission/[0.13]image_caption_pretrained_013.csv'
                       ]

In [29]:
submission_weight = [0.405, 0.352, 0.31, 0.267, 0.23, 0.13]

output_name = 'submission/ensembled_0.405_0.352_0.31_0.267_0.23_0.13_models_average_by_score.csv'

In [30]:
def calc_single(pred_list):
    score_dict = {}
    for i in range(len(pred_list)):
        for j in range(len(pred_list[i])):
            if pred_list[i][j] not in score_dict:
                score_dict[pred_list[i][j]] = 0.0
            score_dict[pred_list[i][j]] += float(len(pred_list[i]) - j) / len(pred_list[i]) * submission_weight[i]
    sorted_score = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)
    return map(lambda x: x[0], sorted_score[:20])

In [31]:
def ensemble():
    res = []
    dfs = [pd.read_csv(file_path, encoding='utf-8') for file_path in submission_file_list]
    for i in range(dfs[0].shape[0]):
        pred_list = [df.iloc[i]['Top_20_Image_IDs'].split(' ') for df in dfs]
        res.append(' '.join(calc_single(pred_list)))
    print len(res)
    return pd.DataFrame({'Descritpion_ID': dfs[0]['Descritpion_ID'].tolist(), 'Top_20_Image_IDs': res})

In [32]:
df_ensembled = ensemble()

2000


In [33]:
print df_ensembled

     Descritpion_ID                                   Top_20_Image_IDs
0             0.txt  1105.jpg 105.jpg 292.jpg 1480.jpg 338.jpg 942....
1             1.txt  833.jpg 818.jpg 80.jpg 589.jpg 457.jpg 314.jpg...
2             2.txt  838.jpg 159.jpg 1724.jpg 1183.jpg 1840.jpg 147...
3             3.txt  1514.jpg 26.jpg 1765.jpg 1215.jpg 1778.jpg 770...
4             4.txt  742.jpg 922.jpg 1231.jpg 292.jpg 594.jpg 985.j...
5             5.txt  771.jpg 51.jpg 492.jpg 1693.jpg 1335.jpg 814.j...
6             6.txt  1855.jpg 1216.jpg 1815.jpg 1660.jpg 1701.jpg 4...
7             7.txt  1285.jpg 168.jpg 689.jpg 448.jpg 409.jpg 975.j...
8             8.txt  419.jpg 103.jpg 719.jpg 1283.jpg 531.jpg 727.j...
9             9.txt  549.jpg 614.jpg 770.jpg 554.jpg 1886.jpg 899.j...
10           10.txt  904.jpg 42.jpg 1421.jpg 634.jpg 1482.jpg 1366....
11           11.txt  814.jpg 492.jpg 1693.jpg 51.jpg 771.jpg 846.jp...
12           12.txt  1653.jpg 1656.jpg 1031.jpg 1950.jpg 837.jpg 40...
13    

In [34]:
df_ensembled.to_csv(output_name, index=False)